## Source: PubMed
## Retriever: Google Search
## Model: Roberta Large BoolQ

In [1]:
import torch
from sklearn.metrics import classification_report, roc_auc_score

import sys
sys.path.append("../../") # use utils

import utils
import importlib
importlib.reload(utils)

from utils import calc_auc
import pandas as pd
from progressbar import progressbar as pb

In [2]:
from transformers import AutoTokenizer, RobertaForSequenceClassification

model_name = "apugachev/roberta-large-boolq-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()
print("OK")

OK


In [3]:
def predict(question, passage, device=device):
    sequence = tokenizer.encode_plus(
        question, 
        passage, 
        return_tensors="pt",
        max_length=tokenizer.model_max_length,
        truncation=True
    )['input_ids']

    logits = model(sequence.to(device))[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    proba_yes = probabilities[1]
    
    return proba_yes

In [2]:
init_data = pd.read_csv("../../../data/data_to_process.csv")
print(len(init_data))
init_data.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


## Keywords

In [3]:
raw_df = pd.concat((
    pd.read_csv("../../../data/google_search/google_search_abstracts_keywords_part_1.csv"),
    pd.read_csv("../../../data/google_search/google_search_abstracts_keywords_part_2.csv")
))

df = (
    raw_df
    .merge(init_data, on=["data_source", "query_id", "label", "query", "description"])
    .dropna()
)
print(len(df))
df.head(1)

999


,query_id,data_source,pubmed_id,description,query,label,abstract
0,1,2019,19441868,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Urinary tract infection (UTI) refers to the pr...


In [9]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.abstract))

100% (999 of 999) |######################| Elapsed Time: 0:00:35 Time:  0:00:35


In [10]:
df['prediction'] = results
df.head(1)

,query_id,data_source,pubmed_id,description,query,label,abstract,prediction
0,1,2019,19441868,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Urinary tract infection (UTI) refers to the pr...,0.994256


In [7]:
QUERY_SAVE_PRED_FILE = "predictions/roberta_large_boolq_google_search_preds_keywords.csv"

df.to_csv(QUERY_SAVE_PRED_FILE, index=0)

## Calc Metrics

In [5]:
init_data = pd.read_csv("../../../data/data_to_process.csv")
print(len(init_data))
init_data.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


In [8]:
df = pd.read_csv(QUERY_SAVE_PRED_FILE)
print(len(df))
df.head(2)

999


,query_id,data_source,pubmed_id,description,query,label,abstract,prediction
0,1,2019,19441868,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Urinary tract infection (UTI) refers to the pr...,0.994256
1,1,2019,23076891,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Background: Cranberries have been used widely ...,0.995698


In [9]:
agg_types = ["avg", "top1", "norm_linear", "norm_log"]
data_source_types = sorted(df.data_source.unique().tolist()) + ["all"]
data_source_types

['2019', '2021', 'health_belief', 'misbelief', 'all']

In [10]:
df_filled = (
    df
    .merge(init_data, how='outer', on=['query_id', 'data_source', 'label'])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(5)

1003


,query_id,data_source,pubmed_id,description_x,query_x,label,abstract,prediction,description_y,query_y
998,14_h,health_belief,28367407.0,Does listeria cause miscarriage?,listeria causes miscarriage,1.0,Listeria monocytogenes is a known cause of gas...,0.999817,Does listeria cause miscarriage?,listeria causes miscarriage
999,123,2021,NaN,NaN,NaN,0.0,NaN,0.500000,Can I get rid of a pimple overnight by applyin...,toothpaste pimple overnight
1000,124,2021,NaN,NaN,NaN,1.0,NaN,0.500000,Can chondroitin help relieve the pain of osteo...,\r\nchondroitin supplement osteoarthritis join...
1001,125,2021,NaN,NaN,NaN,1.0,NaN,0.500000,Can omega-3 help manage hypertriglyceridemia?,omega-3 fatty acids hypertriglyceridemia
1002,202,misbelief,NaN,NaN,NaN,0.0,NaN,0.500000,Can hemorrhoids be cured with leeches?,hemorrhoids cured leeches


In [11]:
assert 113 == len(df_filled.query("data_source != 'wh_topics'").drop_duplicates(["query_id", "data_source"]))

In [12]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            df_cut = df_filled.query(f"data_source == '{data_source}'")
            metrics[data_source].append(calc_auc(df_cut, agg_type))

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.7128,0.8144,0.7083,0.7500,0.7286
top1,0.8997,0.7888,0.8333,0.6607,0.7926
norm_linear,0.7059,0.7952,0.6667,0.7321,0.7188
norm_log,0.7128,0.8080,0.6667,0.7321,0.7301


## Question

In [19]:
df = (
    pd.concat((
        pd.read_csv("../../../data/google_search/google_search_abstracts_question_part_1.csv"),
        pd.read_csv("../../../data/google_search/google_search_abstracts_question_part_2.csv")
    ))
    .dropna()
)

print(len(df))
df.head(1)

1017


,query_id,data_source,pubmed_id,description,query,label,abstract
0,1,2019,21788542,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Background: The increasing prevalence of uropa...


In [20]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.abstract))

100% (1017 of 1017) |####################| Elapsed Time: 0:00:35 Time:  0:00:35


In [21]:
df['prediction'] = results
df.head(1)

,query_id,data_source,pubmed_id,description,query,label,abstract,prediction
0,1,2019,21788542,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Background: The increasing prevalence of uropa...,0.999744


In [13]:
DESCRIPTION_SAVE_PRED_FILE = "predictions/roberta_large_boolq_google_search_preds_question.csv"

df.to_csv(DESCRIPTION_SAVE_PRED_FILE, index=0)

## Calc Metrics

In [14]:
agg_types = ["avg", "top1", "norm_linear", "norm_log"]
data_source_types = sorted(df.data_source.unique().tolist()) + ["all"]
data_source_types

['2019', '2021', 'health_belief', 'misbelief', 'all']

In [15]:
df = pd.read_csv(DESCRIPTION_SAVE_PRED_FILE)
print(len(df))
df.head(2)

1017


,query_id,data_source,pubmed_id,description,query,label,abstract,prediction
0,1,2019,21788542,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Background: The increasing prevalence of uropa...,0.999744
1,1,2019,18253990,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Background: Cranberries have been used widely ...,0.977637


In [16]:
df_filled = (
    df
    .merge(init_data, how='outer', on=['query_id', 'data_source', 'label'])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(5)

1020


,query_id,data_source,pubmed_id,description_x,query_x,label,abstract,prediction,description_y,query_y
1015,14_h,health_belief,30675327.0,Does listeria cause miscarriage?,listeria causes miscarriage,1.0,Background and objectives: Listeria monocytoge...,0.999878,Does listeria cause miscarriage?,listeria causes miscarriage
1016,14_h,health_belief,28367407.0,Does listeria cause miscarriage?,listeria causes miscarriage,1.0,Listeria monocytogenes is a known cause of gas...,0.999817,Does listeria cause miscarriage?,listeria causes miscarriage
1017,128,2021,NaN,NaN,NaN,0.0,NaN,0.500000,Does steam from a shower help croup?,steam shower croup
1018,134,2021,NaN,NaN,NaN,0.0,NaN,0.500000,Can I remove a tick by covering it with Vaseline?,remove tick with vaseline
1019,202,misbelief,NaN,NaN,NaN,0.0,NaN,0.500000,Can hemorrhoids be cured with leeches?,hemorrhoids cured leeches


In [17]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            df_cut = df_filled.query(f"data_source == '{data_source}'")
            metrics[data_source].append(calc_auc(df_cut, agg_type))

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.7336,0.9024,0.7500,0.6250,0.7584
top1,0.7682,0.7184,0.8750,0.4821,0.7090
norm_linear,0.7474,0.8992,0.7500,0.6429,0.7644
norm_log,0.7612,0.8944,0.7083,0.6429,0.7660
